In [9]:
import os 
from xml.dom import minidom
import argparse



def pos_classifier(pos_x, pos_y):
    
    pos_x = int(pos_x)
    pos_y = int(pos_y)   
    sector = 1
    
    if pos_x >= 0 and pos_x <= 166 and pos_y >=0 and pos_y <= 166:
        sector = 1 
    elif pos_x > 166 and pos_x <= 332 and pos_y >=0 and pos_y <= 166:
        sector = 2
    elif pos_x > 332 and pos_x <= 500 and pos_y >=0 and pos_y <= 166:
        sector = 3
    elif pos_x >= 0 and pos_x <= 166 and pos_y >166 and pos_y <= 332:
        sector = 4
    elif pos_x > 166 and pos_x <= 332 and pos_y >166 and pos_y <= 332:
        sector = 5
    elif pos_x > 332 and pos_x <= 500 and pos_y >166 and pos_y <= 332:
        sector = 6
    elif pos_x >= 0 and pos_x <= 166 and pos_y >332 and pos_y <= 500:
        sector = 7
    elif pos_x > 166 and pos_x <= 332 and pos_y >332 and pos_y <= 500:
        sector = 8
    elif pos_x > 332 and pos_x <= 500 and pos_y >332 and pos_y <= 500:
        sector = 9

    
    return str(sector)
  
def pos_classifier_2D(pos_x, pos_y):

    pos_x = int(pos_x)
    pos_y = int(pos_y)

    pos_x = pos_x//50 + 1 
    pos_y = pos_y//50 + 1

    return str(pos_x), str(pos_y)

def get_color(hsl):
    hsl = int(hsl)
    color_list = ['red', 'orange', 'yellow', 'lime', 'green', 'spring_green', 'cyan', 'skyblue','blue', 'purple', 'pink', 'deep_pink', 'red']
    return color_list[hsl//30]

def parse_attribute(pos, polygon, attr_list):

    shape = polygon.getAttribute('class')
    attr_list.append(shape)
    
    # calculate sector 
    pos = pos.replace("translate(", "")
    pos = pos.replace(")", "")
    pos_x = pos.split(',')[0]
    pos_y = pos.split(',')[1]     
    x, y = pos_classifier_2D(pos_x, pos_y)
    attr_list.append(x)
    attr_list.append(y)

    if shape == 'circle':
        radius = polygon.getAttribute('r')
        radius = radius.split('.')[0]
        radius = (str(round(float(radius)/10)*10))
        attr_list.append(radius)
    elif shape == 'rect':
        width = polygon.getAttribute('width')
        height = polygon.getAttribute('height')
        width = (str(round(float(width)/10)*10))
        height = (str(round(float(height)/10)*10))
        attr_list.append(width)
        attr_list.append(height)


    style = polygon.getAttribute('style') 
    style = style.replace("fill: hsl(","").split('.')[0] 
    style = get_color(style)      
    attr_list.append(style)

    #print(attr_list)
    
    return attr_list

def parse_attribute_type2(polygon, attr_list):

    shape = polygon.getAttribute('class') 
    attr_list.append(shape)
    
    style = polygon.getAttribute('style')
    style = style.replace("fill: hsl(","").split(',')[0]
    style = get_color(style)      
    
    if shape == 'circle':
        radius = polygon.getAttribute('r')
        radius = radius.split('.')[0]
        radius = (str(round(float(radius)/10)*10))  
        
        cx = polygon.getAttribute('cx')
        cy = polygon.getAttribute('cy')
        x, y= pos_classifier_2D(cx,cy)
        
        attr_list.append(x)
        attr_list.append(y)
        attr_list.append(radius)
        attr_list.append(style)
        
    elif shape == 'rect':
        x = polygon.getAttribute("x")
        y = polygon.getAttribute("y")
        x, y = pos_classifier_2D(x,y) 
        attr_list.append(x)
        attr_list.append(y)
        
        width = polygon.getAttribute('width')
        height = polygon.getAttribute('height')
        width = (str(round(float(width)/10)*10))
        height = (str(round(float(height)/10)*10))
        
        attr_list.append(width)
        attr_list.append(height)
        attr_list.append(style)


    style = polygon.getAttribute('style') 
    style = style.replace("fill: hsl(","").split('.')[0]      
    #attr_list.append(style)

    #print(attr_list)
    
    return attr_list
    
    

def main():

    svg_path  ='data/nobject_test/svg'
    caption_path = 'data/nobject_test/caption'
    #svg_path  ='data/2object/svg'
    #caption_path = 'data/2object/caption'

    file_list = os.listdir(svg_path)

    tot_list = []
    cnt = 0   
    for f_list in file_list:
        #cnt+=1 
        if cnt>2:
            break; 
        fname = os.path.join(svg_path, f_list)
        doc = minidom.parse(fname)
        svg = (doc.getElementsByTagName('svg'))    
        attr_list = []
        
        if svg[0].firstChild.tagName == 'g':
            g = svg[0].firstChild
            polygon = g.firstChild
            pos = g.getAttribute('transform')
            attr_list = parse_attribute(pos, polygon, attr_list)
       
        else:
            for node in svg[0].childNodes:
                if node.hasAttribute('transform') : 
                    pos = node.getAttribute('transform')
                    attr_list = parse_attribute(pos, node, attr_list)
                else:
                    attr_list = parse_attribute_type2(node, attr_list)
        print(fname)
        print(attr_list)
        
        tot_list.append(attr_list)
        
        
    #print(cnt)
        
        

if __name__ == '__main__':
    main()


data/nobject_test/svg/517.svg
['circle', '2', '8', '70', 'red', 'circle', '10', '2', '40', 'green', 'circle', '10', '10', '40', 'cyan', 'circle', '3', '1', '40', 'purple', 'circle', '5', '2', '60', 'orange', 'circle', '8', '6', '60', 'pink', 'rect', '2', '8', '40', '60', 'skyblue']
data/nobject_test/svg/587.svg
['circle', '7', '6', '30', 'purple', 'circle', '1', '9', '90', 'orange', 'circle', '9', '8', '90', 'skyblue', 'circle', '8', '8', '100', 'lime', 'circle', '8', '3', '110', 'skyblue', 'circle', '1', '3', '80', 'red', 'rect', '9', '2', '40', '90', 'skyblue']
data/nobject_test/svg/649.svg
['circle', '10', '6', '110', 'skyblue', 'rect', '8', '1', '110', '80', 'blue', 'rect', '6', '1', '40', '40', 'pink', 'rect', '3', '7', '90', '90', 'skyblue']
data/nobject_test/svg/654.svg
['circle', '1', '7', '70', 'spring_green', 'circle', '4', '4', '80', 'deep_pink', 'circle', '8', '3', '30', 'orange', 'rect', '1', '2', '110', '90', 'skyblue']
data/nobject_test/svg/771.svg
['rect', '7', '3', '20

In [10]:
arr = []
arr.append('a','b')

In [11]:
arr

[['a', 'b']]

In [ ]:
import numpy as np

c = np.array([[1],[2],[3],[4]])
d = np.array([[1],[2],[3],[4]])
if(np.array_equal(c, d)):
    print('a')